# Imports

In [1]:
from dotenv import load_dotenv
import logging

load_dotenv(dotenv_path=".env.evaluation", verbose=True, override=True)
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import os

from autocommit_evaluation.core.enums import EnvironmentKey
from autocommit_evaluation.cmg.evaluators import CommitMessageGenerator
from autocommit_evaluation.cmg import evaluator
from autocommit_evaluation.core import (
    main_few_shot_high_level_context_cmg_chain,
    main_zero_shot_low_level_context_cmg_chain,
    main_few_shot_low_level_context_cmg_chain,
    main_zero_shot_high_level_context_cmg_chain,
    main_high_level_context_chain,
    baseline_high_level_context_chain,
    baseline_zero_shot_low_level_context_cmg_chain,
    baseline_zero_shot_high_level_context_cmg_chain,
    baseline_few_shot_high_level_context_cmg_chain
)
from autocommit.core.models import CommitDataModel

# Initialization

In [3]:
COMMIT_DATA_JSON_FILE_PATH = os.path.join("autocommit_evaluation", "data", "cmg", "commits.evaluation.json")
CONTEXT_DATA_PATH = os.path.join("autocommit_evaluation","data", "context")

DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.path.join(
    "out", "evaluation", "highlevelcontext"
)
DEFAULT_CMG_OUTPUT_PATH = os.path.join("out", "evaluation", "cmg")
DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH = os.path.join(
    "out", "evaluation", "diffclassification"
)

DIFF_CLASSIFIER_CHAINS = [
    main_zero_shot_low_level_context_cmg_chain,
    main_zero_shot_high_level_context_cmg_chain,
]

HIGH_LEVEL_CONTEXT_CHAINS = [
    main_high_level_context_chain,
    baseline_high_level_context_chain
]

GENERATORS = [
    CommitMessageGenerator(
        "Main Zero-Shot Low-Level Context Generator", main_zero_shot_low_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Main Few-Shot Low-Level Context Generator", main_few_shot_low_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Main Zero-Shot High-Level Context Generator", main_zero_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Main Few-Shot High-Level Context Generator", main_few_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Baseline Few-Shot Low-Level Context Generator", baseline_zero_shot_low_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Baseline Zero-Shot High-Level Context Generator", baseline_zero_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Baseline Few-Shot High-Level Context Generator", baseline_few_shot_high_level_context_cmg_chain
    )
]

In [4]:
HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.getenv(
        EnvironmentKey.HIGH_LEVEL_CONTEXT_OUTPUT_PATH.value,
        DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH,
    )

CMG_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CMG_OUTPUT_PATH.value, DEFAULT_CMG_OUTPUT_PATH
    )

DIFF_CLASSIFICATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.DIFF_CLASSIFICATION_OUTPUT_PATH.value,
        DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH,
    )

In [5]:
def get_commits(path: str) -> list[CommitDataModel]:
        with open(path, "r", encoding="utf-8") as file:
            json_string = file.read()

        return CommitDataModel.from_json(json_string)

COMMITS = get_commits(COMMIT_DATA_JSON_FILE_PATH)

# Classify Diff

In [ ]:
INCLUDED_DIFF_CLASSIFIER_CHAIN_INDICES = [0, 1]

for index in INCLUDED_DIFF_CLASSIFIER_CHAIN_INDICES:
    evaluator.classify_diffs(DIFF_CLASSIFIER_CHAINS[index], COMMITS, CONTEXT_DATA_PATH, DIFF_CLASSIFICATION_OUTPUT_PATH)

# Get High Level Context

In [ ]:
INCLUDED_HIGH_LEVEL_CONTEXT_CHAIN_INDICES = [0]

for index in INCLUDED_HIGH_LEVEL_CONTEXT_CHAIN_INDICES:
    evaluator.get_high_level_contexts(
        HIGH_LEVEL_CONTEXT_CHAINS[index],
        COMMITS, 
        CONTEXT_DATA_PATH, 
        HIGH_LEVEL_CONTEXT_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.smith.langchain.com:443
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Given a Git diff and the relevant source code, write a concise summary of the code changes in a way that a non-technical person can understand. Summarize in maximum three concise sentences. \n\nAvoid adding any additional comments or annotations to the summary.\n\nGit diff:\ndiff --git a/core/src/main/java/kafka/log/remote/RemoteLogManager.java b/core/src/main/java/kafka/log/remote/RemoteLogManager.java\nindex c1c87d579e..3eacbea475 100644\n--- a/core/src/main/java/kafka/log/remote/RemoteLogManager.java\n+++ b/core/src/main/java/kafka/log/remote/RemoteLogManager.java\n@@ -983,7 +983,9 @@ public class RemoteLogManager implements Closeable {\n                     }\n                 }\n                 if (shouldDeleteSegment) {\n-                    logS

DEBUG:urllib3.connectionpool:https://api.smith.langchain.com:443 "POST /runs/multipart HTTP/11" 202 34


# Generate Commit Message

In [ ]:
INCLUDED_GENERATOR_INDICES = [1, 2, 3]
INCLUDED_GENERATOR_INDICES = [2]

filtered_generators = [GENERATORS[i] for i in INCLUDED_GENERATOR_INDICES]
evaluator.evaluate(filtered_generators, COMMITS, CONTEXT_DATA_PATH, CMG_OUTPUT_PATH)